In [1]:
from mdagent.subagents import SubAgentSettings, SubAgentInitializer
from mdagent.utils import PathRegistry

pathregistry = PathRegistry.get_instance()
settings = SubAgentSettings(
    path_registry=pathregistry, subagents_model="gpt-3.5-turbo",temp=0.1
)
initializer = SubAgentInitializer(settings=settings)

In [2]:
## skill agent
example_fxn_code = '''
def get_pdb(query_string, PathRegistry):
    """
    Search RSCB's protein data bank using the given query string
    and return the path to pdb file
    """

    url = "https://search.rcsb.org/rcsbsearch/v2/query?json={search-request}"
    query = {
        "query": {
            "type": "terminal",
            "service": "full_text",
            "parameters": {"value": query_string},
        },
        "return_type": "entry",
    }
    r = requests.post(url, json=query)
    if r.status_code == 204:
        return "No Content Error: PDB ID not found for this substance."
    elif "result_set" in r.json() and len(r.json()["result_set"]) > 0:
        pdbid = r.json()["result_set"][0]["identifier"]
        url = f"https://files.rcsb.org/download/{pdbid}.cif"
        pdb = requests.get(url)
        filename = f"{pdbid}.cif"
        with open(filename, "w") as file:
            file.write(pdb.text)
        return filename
'''

skill_agent = initializer.create_skill_agent()

In [3]:
result = skill_agent.add_new_tool(example_fxn_code)

Function name: get_pdb
Tool name: GetPDB
Tool description: 
Search RSCB's protein data bank using the given query string and return the path to pdb file.

INPUT:
- query_string: a string representing the query to search for in the protein data bank.
- PathRegistry: a path registry object that stores the path to the pdb file.

OUTPUT:
- filename: a string representing the path to the downloaded pdb file.

The function sends a POST request to the RSCB's protein data bank search API with the given query string. If the response status code is 204, it means that no PDB ID was found for the given query and the function returns an error message. If the response contains a result set and the length of the result set is greater than 0, the function retrieves the PDB ID from the response and constructs a download URL for the corresponding CIF file. It then sends a GET request to download the CIF file and saves it with the PDB ID as the filename. The function returns the filename representing the

In [4]:
len(skill_agent.get_skills())